In [60]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv

load_dotenv()

# llm = ChatOpenAI(
#     model="google/gemini-2.0-flash-001",
#     api_key=os.getenv("OPENAI_API_KEY"),
#     base_url=os.getenv("OPENAI_API_BASE"),
#     temperature=0.2,
# )

llm = ChatOllama(model="llama3.2")

In [61]:
from langchain.tools import tool

@tool
def estimate_calories(exercise: str, duration_min: int, weight_kg: float) -> str:
    """This function is used to calculate calories for an exercise. Input the Exercise as string and the duration and the person weight to calculate.
    Currently only supports -> Push Ups, Bench Press and Dumbbell Flyes"""
    met_values = {
        "push ups": 3.8,
        "bench press": 6.0,
        "dumbbell flyes": 4.0,
    }
    met = met_values.get(exercise.lower(), 4.0)
    calories = (met * 3.5 * weight_kg / 200) * duration_min
    return f"Estimated {calories:.1f} calories burned doing {exercise} for {duration_min} mins at {weight_kg}kg."


In [62]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [63]:
from langchain import hub

tools = [estimate_calories, search]

prompt = hub.pull("hwchase17/openai-functions-agent")

In [64]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm,tools,prompt)

In [65]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [66]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

chat_histories = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_key="chat_history",
)

In [67]:
agent_with_chat_history.invoke({"input":"So I wanted to start going to the gym, what is the best exercise for me to do on chest day?"},
                               config={"configurable": {"session_id": "user_a"}})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'best exercise for chest day at the gym'}`


[{'title': '10 Best Chest Exercises for Building Muscle - Bodybuilding.com', 'url': 'https://shop.bodybuilding.com/blogs/training/10-best-chest-exercises-for-building-muscle?srsltid=AfmBOopxtk7-UyKw9DjYAr6Q3ud2KCrgHR6gcxWXrZnZtMnTbKoCMXYz', 'content': "In your workout: If you can do them for high reps, dips make a great finisher to a pro-level chest day. If you can't, you can do them earlier in your session in traditional strength- or muscle-building rep ranges, such as 6-8 or 8-10 reps. Dips also make a great superset pairing with push-ups for a big pump at the end of your workout.\n\nChest Fly [...] All types of dips hit the chest heavily, but on chest day, it's worth making sure you're doing dips that emphasize the pecs. Put your feet up behind you, lean forward as far as possible, and allow your elbows to flare out slightly as you dip.\n\n\n\n

{'input': [HumanMessage(content='So I wanted to start going to the gym, what is the best exercise for me to do on chest day?', additional_kwargs={}, response_metadata={})],
 'output': "Based on the search results, the best exercises for a chest day at the gym are:\n\n1. Bench Press: A compound exercise that targets the middle chest, shoulders, and triceps.\n2. Dips: Can be done with or without weights and target the lower chest and can also be used as a finisher to a pro-level chest day.\n3. Pull-overs: Targets the upper chest and can be an effective variation for building strength and muscle mass.\n4. Flys (with or without weights): A great exercise for targeting the lower chest and can also be used as a finishing move for a chest day workout.\n5. Bear Plank Chest Press: Challenges stability and torches the core, making it an effective exercise for building overall upper body strength.\n\nIt's also recommended to incorporate other exercises such as push-ups, incline bench press, and s

In [40]:
agent_with_chat_history.invoke({"input":"Cool, so which one of these can you calculate the calories for?"})



> Entering new AgentExecutor chain...
I can calculate the calories for Push Ups, Bench Press, and Dumbbell Flyes.


> Finished chain.


{'input': 'Cool, so which one of these can you calculate the calories for?',
 'output': 'I can calculate the calories for Push Ups, Bench Press, and Dumbbell Flyes.\n'}

In [42]:
agent_with_chat_history.invoke({"input":"Cool, i am 75kg and i would bench press for 6 minutes, calculate the calories for that, also for the same metrics calculate the calories if i were to do push ups and compare the calories for me"})



> Entering new AgentExecutor chain...

Invoking: `estimate_calories` with `{'duration_min': 6, 'exercise': 'Bench Press', 'weight_kg': 75}`


Estimated 47.2 calories burned doing Bench Press for 6 mins at 75.0kg.
Invoking: `estimate_calories` with `{'duration_min': 6, 'weight_kg': 75, 'exercise': 'Push Ups'}`


Estimated 29.9 calories burned doing Push Ups for 6 mins at 75.0kg.Okay, so you would burn approximately 47.2 calories doing Bench Press for 6 minutes at 75kg. Alternatively, you would burn approximately 29.9 calories doing Push Ups for 6 minutes at 75kg. Therefore, you would burn more calories doing bench press.


> Finished chain.


{'input': 'Cool, i am 75kg and i would bench press for 6 minutes, calculate the calories for that, also for the same metrics calculate the calories if i were to do push ups and compare the calories for me',
 'output': 'Okay, so you would burn approximately 47.2 calories doing Bench Press for 6 minutes at 75kg. Alternatively, you would burn approximately 29.9 calories doing Push Ups for 6 minutes at 75kg. Therefore, you would burn more calories doing bench press.\n'}

In [51]:
agent_with_chat_history.invoke({"input":"Hi, I am Youssef"},
                               config={"configurable": {"session_id": "user_a"}})

agent_with_chat_history.invoke({"input":"Hi, I am Ahmed"},
                               config={"configurable": {"session_id": "user_b"}})

agent_with_chat_history.invoke({"input":"What is my Name?"},
                               config={"configurable": {"session_id": "user_a"}})

agent_with_chat_history.invoke({"input":"What is my Name?"},
                               config={"configurable": {"session_id": "user_b"}})



> Entering new AgentExecutor chain...
Hi Youssef, nice to meet you. How can I help you today?


> Finished chain.


> Entering new AgentExecutor chain...
Hello Ahmed, how can I help you today?


> Finished chain.


> Entering new AgentExecutor chain...
Your name is Youssef.


> Finished chain.


> Entering new AgentExecutor chain...
Your name is Ahmed.


> Finished chain.


{'input': [HumanMessage(content='Hi, I am Ahmed', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ahmed, how can I help you today?\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my Name?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Ahmed.\n'}